In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd drive/MyDrive/IW06-07/deepxi

In [ ]:
!pip3 install jiwer

In [ ]:
import pickle
from jiwer import wer, cer, RemoveWhiteSpace

In [ ]:
trans = []
with open('deepxi_trans1long.pickle', 'rb') as e:
  trans.append(pickle.load(e))
with open('deepxi_trans1medium.pickle', 'rb') as e:
  trans.append(pickle.load(e))
with open('deepxi_trans1short.pickle', 'rb') as e:
  trans.append(pickle.load(e))

trans_dict = {}
trans_dict['long'] = trans[0]
trans_dict['medium'] = trans[1]
trans_dict['short'] = trans[2]

In [ ]:
trans_dict = {}
with open('deepxi_trans41_finetuned_normal.pickle', 'rb') as e:
  trans_dict = pickle.load(e)

In [ ]:
def calc_stat(stat):  
  total_stat = {}
  num_stat = 2
  total_stat['long-long'] = [0]*num_stat
  total_stat['long-medium'] = [0]*num_stat
  total_stat['long-short'] = [0]*num_stat
  total_stat['medium-long'] = [0]*num_stat
  total_stat['medium-medium'] = [0]*num_stat
  total_stat['medium-short'] = [0]*num_stat
  total_stat['short-long'] = [0]*num_stat
  total_stat['short-medium'] = [0]*num_stat
  total_stat['short-short'] = [0]*num_stat
  # long, medium, short
  total_stat['original-ground'] = [[0, 0, 0], [0,0,0]]
  for i, length in enumerate(trans_dict):
    for audio in trans_dict[length]:
      base = trans_dict[length][audio]['base']
      ori = trans_dict[length][audio]['ori']
      adv = trans_dict[length][audio]['adv']
      enh = trans_dict[length][audio]['enh']
      enh_ori = trans_dict[length][audio]['enh_ori']
      # total_stat['original-ground'][0][i] += stat(base, ori)
      # total_stat['original-ground'][1][i] += stat(base, enh_ori)
      for j, l in enumerate(['long', 'medium', 'short']):
        key = length+'-'+l
        # add_stat = [stat(ori, enh_ori), stat(ori, adv[j]), stat(ori, enh[j]), stat(enh[j], adv[j])]
        add_stat = [stat(ori, adv[j]), stat(ori, enh[j])]
        for idx in range(len(total_stat[key])):
          total_stat[key][idx] += add_stat[idx]

  return total_stat

#Calculate WER

In [ ]:
t = calc_stat(wer)

##Calculate CER

In [ ]:
t = calc_stat(cer)

##Calculate Percentage Decrease

In [ ]:
for key in t:
  print(key)
  inc = (t[key][0]-t[key][1])/t[key][0]
  print(inc*100)

In [ ]:
len(trans_dict['short'])